#Integrantes:
- Natalia Maury Castañeda (u201816996)
- Joaquín Galván
- Carlos Iparraguirre (u201810601)

#Librerías

In [ ]:
import random
import numpy as np
import math

from numpy.random import choice
from timeit import default_timer as timer

#Crear las clases para los agentes

In [ ]:
#Cantidad de tipos de agentes:
n_restaurantes = 300
n_ciudadanos = 2000
n_repartidores = 100

In [ ]:
id_restaurantes = [_ for _ in range(n_restaurantes)]

In [ ]:
class Pedido:
  def __init__(self, id, id_restaurantes, id_ciudadano): #Constructor
    self.id = id
    self.restaurante = random.choice(id_restaurantes)
    self.duracion = random.randint(1,40)
    self.preparando = 0
    self.ciudadano = id_ciudadano
  
  def SetRestaurante(self, rest):
    self.restaurante = rest

In [ ]:
class Agent:
  def __init__(self, id, x, y):  #Constructor
    self.x = x
    self.y = y
    self.id = id

  def update(self):
    pass
    
  def decide(self):
    pass

In [ ]:
def Distancia(x1,y1,x2,y2):
  d = math.pow((x2 - x1),2) + math.pow((y2 - y1),2)
  return d

In [ ]:
def RecibirOrden():
  global agentList
  id = 0
  return id

In [ ]:
class Repartidor(Agent):
  def __init__(self, id):
    x = random.randint(0,400)
    y = random.randint(0,400)
    super().__init__(id,x,y)
    self.id_rest = -1 #id restaurante al que va a ir
    self.pedido = -1 #id del ciudadano que realizó el pedido
    self.muerte_subita = False
    self.mover = False
    self.distancias = [0 for i in range(n_restaurantes)]
    self.dx = 3
    self.dy = 3
    self.x_obj_restaurante = 0
    self.y_obj_restaurante = 0
    self.x_obj_cliente = 0
    self.y_obj_cliente = 0
  
  def MostrarValores(self):
    print('Soy repartidor',self.id,'y mis coordenadas son: X',self.x, 'Y: ', self.y)

  def PenaMuerte(status):
    if status == True:
      muerte_subita = True
    
    return muerte_subita

  def update(self):
    global agentList

    if self.muerte_subita == True:
      print("El repartidor ", self.id, " fue ejecutado")
    
    if self.mover == True:
      while(self.x != self.x_obj_restaurante and self.y != self.y_obj_restaurante):
        self.x += self.dx
        self.y += self.dy
      print("El repartidor ", self.id, "llego al restaurante")
      id_c = agentList.index(self.id_rest).ordenes_terminadas[0].id_ciudadano
      self.x_obj_cliente = agentList.index(id_c).x
      self.y_obj_cliente = agentList.index(id_c).y
      print("El repartidor ", self.id, " irá a la casa del ciudadano ", id_c)
      print("El restaurante ", self.id_rest, "le entregó la orden ", agentList.index(self.id_rest).ordenes_terminadas[0].id, " al repartidor ", self.id)
      agentList.index(self.id_rest).ordenes_terminadas.remove(agentList.index(self.id_rest).ordenes_terminadas.remove[0]) #Eliminar la orden a entregar de ordenes_terminadas
      #agentList.index(self.id_rest).update() #Actualizar lo de restaurante 
      
      while(self.x != self.x_obj_cliente and self.y != self.y_obj_cliente):
        self.x += self.dx
        self.y += self.dy
      print("El repartidor ", self.id, "entrego el pedido")
      agentList.index(id_c).recibido = True
      self.mover = False


  def decide(self):
    global agentList
    restaurantes = []
    x_rest = []
    y_rest = []
    id_rest = []
    j = 2000 #n_ciudadanos
    k = n_restaurantes + n_ciudadanos

    for j in range(k):
      #print("Valor j: ", j) #Solo para corregir un bug
      if j >= 2000 and j < 2300:
        restaurantes.append(agentList[j])
        #print("Soy ", restaurantes[j].id)

    for h in restaurantes:
      print(h.id)
      if h.necesita_repartidor == False:
        print("El restaurante ", h.id,  "fue eliminado del arreglo local porque no necesita repartidor en este momento")
        restaurantes.remove(h)

    for i in range(len(self.distancias)):
      for j in range(len(restaurantes)):
        x1 = restaurantes[j].x
        y1 = restaurantes[j].y

        x2 = self.x
        y2 = self.y
        self.distancias[i] = Distancia(x1, y1, x2, y2)
        x_rest.append(x1)
        y_rest.append(y1)
        id_rest.append(restaurantes[j].id)

    print("La distancia mínima/restaurante más cercano es: ", min(self.distancias))
    minimo = self.distancias.index(min(self.distancias))
    print("Su posición en el vector distancias es: ", minimo)

    self.x_obj_restaurante = x_rest[minimo]
    self.y_obj_restaurante = y_rest[minimo]
    self.id_rest = id_rest[minimo]
    print("El repartidor ", self.id, "irá al restaurante ", self.id_rest, "ubicado en: X = ", self.x_obj_restaurante, ", Y = ", self.y_obj_restaurante)

In [ ]:
class Restaurante(Agent):
  def __init__(self, id):
    x = random.randint(0,400)
    y = random.randint(0,400)
    super().__init__(id,x,y)

    self.max_pedidos = 10
    self.max_pedidos_terminados = random.randint(20,30)
    self.necesita_repartidor = False
    self.ordenes = 0 #Cantidad total de órdenes (terminadas y en proceso)
    self.ordenes_proceso = [] #Vector con las órdenes en proceso
    self.ordenes_terminadas = [] #Vector con las órdenes terminadas esperando repartidor
    self.aceptar_orden = True #Si acepta más órdenes o no
  
  def MostrarValores(self):
    print('Soy restaurante',self.id,' y mis coordenadas son: X',self.x, ' Y: ', self.y, ' mis ordenes son: ', self.ordenes, 
          ' mis ordenes terminadas son: ', len(self.ordenes_terminadas), ' y mis ordenes en proceso son: ', len(self.ordenes_proceso))  
    
  def update(self):
    global agentList
    for i in self.ordenes_proceso:
      if i.preparando != i.duracion:
        i.preparando += 1

      else:
        self.ordenes_proceso.remove(i)
        self.ordenes_terminadas.append(i)
    
    ordenes = len(self.ordenes_proceso) + len(self.ordenes_terminadas)

    if len(self.ordenes_terminadas) >= 1:
      self.necesita_repartidor = True
    
    if len(self.ordenes_terminadas) == 0:
      self.necesita_repartidor = False



  def decide(self):
    if len(self.ordenes_terminadas) == self.max_pedidos_terminados:
      self.aceptar_orden = False
      print("Capacidad máxima, no se aceptan más pedidos")

    elif len(self.ordenes_proceso) == self.max_pedidos:
      self.aceptar_orden = False
    
    else:
      self.aceptar_orden = True
      print("orden aceptada por el restaurante: ", self.id)

In [ ]:
def AsignarHambre(tipo):
  h = 0
  if tipo == 0:
    h = 200
    
  else:
    h = 50
  
  return h

In [ ]:
def RealizarPedido(Pedido, id_restaurantes, id_ciudadano):
   global agentList
   Pedido.__init__(id_restaurantes, id_ciudadano)

   id_res = agentList.index(Pedido.id_restaurantes)
   
   while(True):
     if agentList.index(id_res).aceptar_orden == False:
       agentList.index(id_res).decide()

       if agentList.index(id_res).aceptar_orden == True:
         break

       else:
        id_res = random.choice(id_restaurantes)
        continue
     
     if agentList.index(id_res).aceptar_orden == True:
       agentList.index(id_res).ordenes_proceso.append(Pedido)
       break

   return Pedido

In [ ]:
class Ciudadano(Agent):
  def __init__(self, id):
    x = random.randint(0,400)
    y = random.randint(0,400)
    self.tipo = random.randint(0,1) #0 = poco consumidor, 1 = ultra consumidor
    super().__init__(id,x,y)
    self.hambre = AsignarHambre(self.tipo) #Cada cuánto tiempo realizará un pedido
    self.esperando = False #No realizará acciones si está esperando un pedido: False = no espera pedido; True = espera un pedido
    self.recibido = False
    self.cont = 0 #Tiempo

  def MostrarValores(self):
    print('Soy ciudadano',self.id,' y mis coordenadas son: X',self.x, ' Y: ', self.y, ' mi tipo es: ', self.tipo)  
  
  def update(self):
    global agentList
    if self.esperando == False:
      self.cont += 1

    

    if self.recibido == True:
      print("El ciudadano ", self.id, " recibió su pedido")
  
  def decide(self):
    if self.cont == self.hambre:
      p = RealizarPedido(Pedido, id_restaurantes, self.id)
      print("El ciudadano ", self.id, " realizo un pedido al restaurante ", p.restaurante)
      self.esperando = True
      self.cont = 0

#MAS

In [ ]:
agentList = []

In [ ]:
#for i in range(n_ciudadanos):
  #c = Ciudadano(i)
  #agentList.append(c)

In [ ]:
#j = n_ciudadanos
#n = n_ciudadanos + n_restaurantes

#for j in range(n):
  #r = Restaurante(j)
  #agentList.append(r)

In [ ]:
#k = n_repartidores
#k = n_ciudadanos + n_restaurantes
#m = n_ciudadanos + n_restaurantes + n_repartidores

#for k in range(m):
  #rep = Repartidor(k)
  #agentList.append(rep)

##Probar solución

In [ ]:
#Intentar solución:
z = n_ciudadanos + n_restaurantes + n_repartidores

for i in range(z):
  if i < 2000:
    c = Ciudadano(i)
    agentList.append(c)
  
  elif i >= 2000 and i < n_ciudadanos + n_restaurantes:
    r = Restaurante(i)
    agentList.append(r)
  
  else: #i >= n_ciudadanos + n_restaurantes && i < z
    rep = Repartidor(i)
    agentList.append(rep)

In [ ]:
tick = 0
while(tick <= 5):
  for a in agentList:
    a.MostrarValores()
    a.update()
    a.decide()

  #for a in agentList:
    #if a.muerte_subita == True:
      #agentList.remove(a)

  tick += 1

Se han truncado las últimas 5000 líneas del flujo de salida.
El restaurante  2166 fue eliminado del arreglo local porque no necesita repartidor en este momento
2168
El restaurante  2168 fue eliminado del arreglo local porque no necesita repartidor en este momento
2170
El restaurante  2170 fue eliminado del arreglo local porque no necesita repartidor en este momento
2172
El restaurante  2172 fue eliminado del arreglo local porque no necesita repartidor en este momento
2174
El restaurante  2174 fue eliminado del arreglo local porque no necesita repartidor en este momento
2176
El restaurante  2176 fue eliminado del arreglo local porque no necesita repartidor en este momento
2178
El restaurante  2178 fue eliminado del arreglo local porque no necesita repartidor en este momento
2180
El restaurante  2180 fue eliminado del arreglo local porque no necesita repartidor en este momento
2182
El restaurante  2182 fue eliminado del arreglo local porque no necesita repartidor en este momento
2184
El 